In [1]:
import os
os.chdir('../')

In [2]:
%pwd

'd:\\pythonProjects\\careGuideAI'

# read the documents

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
# extract test from PDF file 
def load_pdf_files(data_path):
    loader=DirectoryLoader(
        data_path,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents=loader.load()
    return documents


In [5]:
extracted_data=load_pdf_files('data')

In [6]:
len(extracted_data)

872

In [7]:
# filter only important data 
from typing import List 
from langchain.schema import Document 

def filter_to_minimal_docs(docs: List[Document])-> List[Document]:
    """ 
    Given a list of document objects, return a new list of Document objects 
    containing only 'source' in metafata and the original page_content
    """
    minimal_docs: List[Document]=[]
    for doc in docs:
        src=doc.metadata.get('source')
        print(src)
        minimal_docs.append(
            Document(page_content=doc.page_content,
                     metadata={'source':src}
                     )

        )
    return minimal_docs 

In [8]:
docs=filter_to_minimal_docs(extracted_data)
len(docs)

data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medical_book.pdf
data\Medic

872

In [9]:
# split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunks=text_splitter.split_documents(minimal_docs)
    return texts_chunks

In [10]:
texts_split=text_split(docs)
print(len(texts_split))

6610


# Azure open AI 

In [11]:
import os 
from openai import AzureOpenAI 

client=AzureOpenAI(
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    api_key=os.getenv('AZURE_OPENAI_KEY'),
    api_version='2024-12-01-preview'
)

response=client.chat.completions.create(
    model='gpt-4o',
    response_format={'type':'json_object'},
    messages=[
        {'role':'system','content':'Your are a helpful assistant designed to output JSON. '},
        {'role':'user','content':'who won the world series in 2020?'}
    ]

)

print(response.choices[0].message.content)

{
  "year": 2020,
  "winner": "Los Angeles Dodgers",
  "runner_up": "Tampa Bay Rays",
  "result": "Los Angeles Dodgers defeated Tampa Bay Rays in 6 games (4–2)"
}


# create embeddings for the model

## Azure open AI embeddings 

In [12]:
from langchain_openai import AzureOpenAIEmbeddings

emb=AzureOpenAIEmbeddings(
    model='text-embedding-3-large',
    azure_deployment='text-embedding-3-large',
    openai_api_type='',
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    api_key=os.getenv('AZURE_OPENAI_KEY')
)

vec=emb.embed_query('hello world')
print(len(vec),vec[:8])

3072 [-0.002786265453323722, -0.022649090737104416, 0.005106728989630938, 0.026939596980810165, 0.015298819169402122, 0.009777569212019444, -0.005585351027548313, 0.08546826243400574]


In [13]:
# generate vector embeddings
# import os 
# from openai import AzureOpenAI
# from dotenv import load_dotenv

# load_dotenv()

# client=AzureOpenAI(
#     api_version='2024-12-01-preview',
#     azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
#     api_key=os.getenv('AZURE_OPENAI_KEY')
# )

In [14]:
# resp = client.embeddings.create(
#     model="text-embedding-3-large",   # your deployment's model ID
#     input="What is my purpose?"
# )

In [15]:
# vec=resp.data[0].embedding
# print(len(vec),vec[:8])

In [16]:
# print(resp)

# langchain hugging face embeddings for backup
- free version

In [17]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """ 
    Download and return the Higging Face embeddings model 
    """
    model_name="sentence-transformers/all-MiniLM-L6-v2"
    embeddings=HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings 

embedding=download_embeddings()

C:\Users\26amr\AppData\Local\Temp\ipykernel_25552\3403150667.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(
d:\anaconda\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
vector=embedding.embed_query('Hello world')
vector

[-0.03447723388671875,
 0.03102317824959755,
 0.0067349825985729694,
 0.02610897831618786,
 -0.03936202824115753,
 -0.16030246019363403,
 0.06692395359277725,
 -0.006441435310989618,
 -0.04745045676827431,
 0.01475885882973671,
 0.07087531685829163,
 0.055527638643980026,
 0.019193371757864952,
 -0.026251353323459625,
 -0.010109566152095795,
 -0.026940438896417618,
 0.022307418286800385,
 -0.022226635366678238,
 -0.1496925950050354,
 -0.017493007704615593,
 0.0076762898825109005,
 0.054352208971977234,
 0.003254421055316925,
 0.03172585740685463,
 -0.0846213698387146,
 -0.029406024143099785,
 0.05159556120634079,
 0.04812407121062279,
 -0.0033147986978292465,
 -0.05827919393777847,
 0.04196927696466446,
 0.022210704162716866,
 0.128188818693161,
 -0.02233891934156418,
 -0.0116562619805336,
 0.06292837858200073,
 -0.032876309007406235,
 -0.09122603386640549,
 -0.03117534890770912,
 0.05269957333803177,
 0.04703480750322342,
 -0.08420306444168091,
 -0.030056176707148552,
 -0.020744828507

In [19]:
print(len(vector))

384


# pinecone integration 

In [20]:
PINECONE_API_KEY=os.getenv('PINECONE_API_KEY')

In [21]:
from pinecone import Pinecone 

pc=Pinecone(api_key=PINECONE_API_KEY)

In [ ]:
# create index for pinecone
from pinecone import ServerlessSpec 
index_name='medical-chatbot'

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=3072, # 384 # higher the dimension the better 
        metric='cosine',
        spec=ServerlessSpec(cloud='aws',region='us-east-1')
    )

index=pc.Index(index_name)

In [23]:
from langchain_pinecone import PineconeVectorStore 

docsearch=PineconeVectorStore.from_documents(
    documents=texts_split,
    embedding=emb,
    index_name=index_name,
    batch_size=50,                # upsert ~50 vectors per request (tune 25–200)
    embeddings_chunk_size=64,
)

# load existing index


In [24]:
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=emb
)

# add more data to the existing pinecone index

In [25]:
add=Document(
    page_content='amruth is an AI god, he created the AGI loop',
    metadata={'source':'year-2070'}
)

In [26]:
docsearch.add_documents(documents=[add])

['edb7a2f1-646b-4916-b760-491f76a641fb']

In [27]:
retriever=docsearch.as_retriever(search_type='similarity',search_kwargs={'k':3})

In [28]:
retrieverd_docs=retriever.invoke('what is Acne? and how do i treat it?')
retrieverd_docs

[Document(id='773b4a72-ecb3-4511-9bd2-c149b82b0808', metadata={'source': 'data\\Medical_book.pdf'}, page_content='isotretinoin. It can be controlled by proper treatment,\nwith improvement taking two or more months. Acne\ntends to reappear when treatment stops, but spontaneous-\nly improves over time. Inflammatory acne may leave\nscars that require further treatment.\nPrevention\nThere are no sure ways to prevent acne, but the fol-\nlowing steps may be taken to minimize flare-ups:\n• gentle washing of affected areas once or twice every day\n• avoid abrasive cleansers\n• use noncomedogenic makeup and moisturizers'),
 Document(id='60d0423a-07fb-40b9-a7de-ed6ba22a1395', metadata={'source': 'data\\Medical_book.pdf'}, page_content='isotretinoin. It can be controlled by proper treatment,\nwith improvement taking two or more months. Acne\ntends to reappear when treatment stops, but spontaneous-\nly improves over time. Inflammatory acne may leave\nscars that require further treatment.\nPreventi

In [29]:
from langchain_openai import AzureChatOpenAI 
from langchain_core.messages import HumanMessage 
llm=AzureChatOpenAI(
    azure_deployment='gpt-4o',
    api_version='2024-12-01-preview',
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    api_key=os.getenv('AZURE_OPENAI_KEY')
)

resp=llm.invoke([HumanMessage(content='Give me 3 fun facts about texas')])

In [30]:
resp

AIMessage(content='Sure! Here are three fun and interesting facts about Texas:\n\n1. **Six Flags Over Texas:** Texas is famously known as the "Six Flags State" because six different nations have had sovereignty over the region at some point in history. These nations include Spain, France, Mexico, the Republic of Texas (independent nation), the Confederate States of America, and the United States. The Six Flags amusement parks even got their name from this historic detail!\n\n2. **Bigger is Better:** Texas is the second-largest state in the U.S., both in size and population (only Alaska and California beat it in these respective categories). To put it in perspective, Texas is so massive that if it were its own country, it would be the 39th largest in the world—larger than every European country except Russia!\n\n3. **Official State Snack:** Texans are proud of their food, and chips and salsa hold the high honor of being the official state snack of Texas! Thanks to its rich history and i

In [31]:
# chain 
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain 
from langchain_core.prompts import ChatPromptTemplate 

In [32]:
system_prompt=(
    "You are an assistant for question-answering task"
    "Use the following pieces of retrieved context to answer"
    "the question. If you dont know the answer say that you dont know"
    " Use three sentences maximum and keep the answer concise"
    "\n\n"
    "{context}"
)

promp=ChatPromptTemplate.from_messages(
    [
        ('system',system_prompt),
        ('human',"{input}"),
        
    ]
)

In [33]:
# create chain and qa chain 
question_answer_chain=create_stuff_documents_chain(llm, promp)
rag_chain=create_retrieval_chain(retriever, question_answer_chain)

In [34]:
response=rag_chain.invoke({'input':'what is Acromegaly and gigantism'})
print(response['answer'])

Acromegaly is a disorder caused by the abnormal release of a chemical from the pituitary gland, leading to increased growth in bones, soft tissues, and various disturbances in the body. Gigantism occurs when this abnormal growth happens during childhood, leading to excessive height and size. Acromegaly typically occurs after the growth plates in bones have closed, resulting in enlarged features rather than height increase.


In [35]:
response=rag_chain.invoke({'input':'what is Acne?'})
print(response['answer'])

Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when skin pores become clogged with oil, dead skin cells, and bacteria. The medical term for common acne is acne vulgaris.


In [36]:
response=rag_chain.invoke({'input':'who is amruth?'})
print(response['answer'])

Amruth is referred to as an "AI god" and is credited with creating the AGI loop.
